In [2]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [3]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [4]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [8]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×12 TimeArray{Any, 2, Date, Matrix{Any}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       23.77 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       24.37 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       24.96 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.54 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.57 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       25.29 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.14 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.35 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮     │   

In [17]:
freq = 4
returns = percentchange(data_source[:Total_Stock], :log)
macro_data = percentchange(data_source[:CPI, :INDPRO], :log)
real_returns = returns .- macro_data[:CPI]

#dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])
#term = data_source[:Yeld_10Y] .- data_source[:T_Bill] 

rates = log1p.(data_source[:T_Bill, :Yeld_10Y] ./ freq) 

data = merge(real_returns, macro_data, rates)

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq))


,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1613,0.7639,0.0657,0.2676,-1.2903,-0.9104,-0.1069,0.0681,1.598,0.4072,0.1152
CPI,0.0173,0.1612,0.0353,0.0519,-0.1016,0.4804,0.0139,0.4625,1.7531,2.0398,0.0306
INDPRO,0.0381,0.2351,0.0243,0.0637,-0.2929,-1.024,-0.0032,0.365,3.3108,0.6366,0.0294
T_Bill,0.0152,0.152,0.0422,0.0567,0.0001,0.818,0.0198,0.9526,1.0747,2.7682,0.0424
Yeld_10Y,0.0143,0.1553,0.0557,0.0734,0.0066,0.8725,0.0371,0.9765,0.6079,3.8845,0.0494


nothing

In [18]:
assets_names = colnames(s_data)
T, n_assets = size(s_data)
model = VARModel(s_data)
model_summary(model)

,const,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y,std
Total_Stock_CPI,0.0244,0.0444,-1.4347,-0.2493,-0.2284,0.5692,0.0802
CPI,0.0035,0.0016,0.3229,0.0302,0.5166,-0.2262,0.0074
INDPRO,-0.0009,0.0794,0.1405,0.3459,-1.1093,0.9917,0.0161
T_Bill,-0.0001,0.0027,0.0285,0.0151,0.8654,0.0845,0.0022
Yeld_10Y,0.0004,0.0019,0.0265,0.004,0.044,0.9213,0.0014


,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y
Total_Stock_CPI,1.0,-0.14,0.06,-0.09,-0.05
CPI,-0.14,1.0,-0.01,0.28,0.35
INDPRO,0.06,-0.01,1.0,0.34,0.27
T_Bill,-0.09,0.28,0.34,1.0,0.62
Yeld_10Y,-0.05,0.35,0.27,0.62,1.0


In [19]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y
1,0.075,0.0274,-0.0172,0.0468,0.042
5,0.0676,0.0338,0.0087,0.0406,0.0474
15,0.0662,0.0347,0.0178,0.041,0.052
25,0.0656,0.0353,0.0196,0.0421,0.054


,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y
1,0.145,0.017,0.0376,0.0083,0.0055
5,0.0742,0.0135,0.0218,0.0156,0.0133
15,0.0442,0.011,0.0129,0.0176,0.017
25,0.0346,0.0099,0.0102,0.017,0.0169


,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y
1,0.0045,0.01,-0.0082,-0.0491,-0.0269
5,-0.0024,0.0265,-0.0113,0.0243,0.0257
15,0.0266,0.0482,0.0208,0.0346,0.0394
25,-0.024,0.0755,0.0084,0.0645,0.0614


,Total_Stock_CPI,CPI,INDPRO,T_Bill,Yeld_10Y
1,-0.0445,0.0067,0.0055,0.0163,0.0214
5,0.0406,-0.0135,-0.026,-0.0267,-0.0652
15,0.0466,-0.0044,0.0021,0.0037,-0.0024
25,-0.0595,0.068,0.0126,0.0521,0.0413


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.261,-0.2072,-0.1632,-0.0243,0.0745,0.173,0.313,0.3567,0.4067
5,-0.1064,-0.0764,-0.0551,0.0175,0.0682,0.1177,0.1903,0.215,0.2433
15,-0.0358,-0.02,-0.0062,0.0362,0.0665,0.0958,0.1386,0.1532,0.1708
25,-0.0143,-0.002,0.0078,0.0424,0.0658,0.0892,0.122,0.132,0.146


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0122,-0.0063,-0.0005,0.0161,0.0273,0.039,0.0553,0.0606,0.0667
5,0.0034,0.0078,0.0119,0.0245,0.0337,0.043,0.0562,0.0604,0.0654
15,0.0091,0.0131,0.0168,0.0271,0.0346,0.0421,0.053,0.0562,0.0611
25,0.0129,0.0161,0.0193,0.0286,0.0351,0.0418,0.0519,0.0547,0.0588


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.105,-0.0906,-0.0794,-0.0427,-0.017,0.0077,0.0451,0.0563,0.0705
5,-0.0422,-0.0342,-0.0271,-0.0061,0.0087,0.0234,0.0442,0.0512,0.06
15,-0.0122,-0.0072,-0.0032,0.009,0.0177,0.0265,0.0393,0.0433,0.0481
25,-0.004,-0.0006,0.003,0.0127,0.0196,0.0265,0.0364,0.0399,0.0432


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.027,0.0302,0.0329,0.0413,0.0469,0.0524,0.0604,0.0631,0.0661
5,0.0052,0.0104,0.0153,0.0301,0.0405,0.0511,0.0661,0.0716,0.0772
15,0.0007,0.0068,0.0122,0.029,0.041,0.0528,0.0704,0.0759,0.0833
25,0.0029,0.0093,0.0144,0.0308,0.042,0.0534,0.0703,0.0759,0.0833


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.0289,0.0311,0.0328,0.0383,0.042,0.0457,0.051,0.0526,0.0548
5,0.0171,0.0218,0.0258,0.0383,0.0473,0.0565,0.0693,0.0732,0.078
15,0.0133,0.0189,0.024,0.0405,0.0519,0.0634,0.0803,0.0857,0.092
25,0.0152,0.0213,0.0267,0.0427,0.0539,0.0652,0.0819,0.0876,0.0944


In [22]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')



:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.221012  0.312033  0.381092  0.467548  0.653697

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 11.0  19.0  27.0  40.0  79.0